In [1]:
# Paper: Duan and Wang. "Adaptive and Robust Multi-Task Learning." arXiv preprint arXiv:2202.05250 (2022).

# Multi-task logistic regression by ARMUL and baseline procedures
# Real dataset: Human activity recognition [Anguita et al. "A public domain dataset for human activity recognition using smartphones." Proceedings of the 21th international European symposium on artificial neural networks, computational intelligence and machine learning. (2013)]
# 30 volunteers (tasks), 225 - 328 samples for each;
# 561 features with time and frequency domain variables;
# 6 activities: walking, walking-upstairs, walking-downstairs, sitting, standing, laying.

# Preprocessing:
# Hold out 20% of data from each task for testing;
# Reduce the dimension to 100 by PCA;
# Convert to binary classification with 2 classes: sitting vs. others.

# Evaluation metric: average misclassification error over all testing data

# Setup
import numpy as np
from ARMUL import ARMUL, Baselines
from preprocessing import load, split

m = 30 # number of tasks
prop = 0.2 * np.ones(m) # hold out 20% of data from each task for testing
eta = 0.1 # step size for the optimization algorithm (proximal gradient descent)
T = 200 # number of iterations in optimization
seed = 10000 # random seed

[data_raw, _, _] = load(dataset = 'HAR_binary', path = '', PCs = 100) # load data
[data_train, data_test] = split(data = data_raw, prop = prop, seed = seed) # train-test split
d = data_train[0][0].shape[1] # dimension
n_list = np.array( [data_train[0][j].shape[0] for j in range(m)] ) # list of sample sizes

In [2]:
# ARMUL
test = ARMUL('logistic')

# Example: take lambda_j = 0.1 * np.sqrt(d / n_j) for all j in [m]
lbd = 0.1 * np.sqrt(d / n_list)

# Vanilla ARMUL
test.vanilla(data_train, lbd, eta_global = eta, eta_local = eta, T_global = T)
test.results = test.evaluate(data_test, model = 'vanilla')
print( 'Vanilla ARMUL: {}'.format(test.results['average error']) ) # average testing error


# Clustered ARMUL
K = 4 # number of clusters
test.clustered(data_train, lbd, K = K, eta_B = eta, eta_local = eta, T_global = T)
test.results = test.evaluate(data_test, model = 'clustered')
print( 'Clustered ARMUL: {}'.format(test.results['average error']) )


# Low-rank ARMUL
K = 4 # rank
test.lowrank(data_train, lbd, K = K, eta_B = eta, eta_Z = eta, eta_local = eta, T_global = T)
test.results = test.evaluate(data_test, model = 'lowrank')
print( 'Low-rank ARMUL: {}'.format(test.results['average error']) )



Vanilla ARMUL: 0.015632633121641426
Clustered ARMUL: 0.01172447484123107
Low-rank ARMUL: 0.008304836345872008


In [3]:
# Cross-validation
C_list = [0.05, 0.1, 0.2, 0.4]
lbd_list = [C * np.sqrt(d / n_list) for C in C_list]
n_fold = 5 # number of folds for cross-validation


print('Vanilla ARMUL') # Vanilla ARMUL
test.cv(data_train, lbd_list, model = 'vanilla', n_fold = 5, seed = seed, eta_global = eta, eta_local = eta, eta_B = eta, eta_Z = eta, T_global = T)
print( 'CV errors corresponding to all C\'s: {}'.format(test.errors_cv) )
test.results = test.evaluate(data_test, model = 'vanilla')
print( 'Average testing error of the final model: {}'.format(test.results['average error']) )



Vanilla ARMUL
CV errors corresponding to all C's: [0.0125908  0.01162825 0.01077827 0.01903212]
Average testing error of the final model: 0.013190034196384953


In [4]:
# Baselines
base = Baselines('logistic')

# Single-task learning
base.STL_train(data_train, eta = eta, T = T)
base.results_baseline = base.evaluate(data_test, model = 'STL')
print( 'Single-task learning: {}'.format(base.results_baseline['average error']) )


# Data pooling
base.DP_train(data_train, eta = eta, T = T)
base.results_baseline = base.evaluate(data_test, model = 'DP')
print( 'Data pooling: {}'.format(base.results_baseline['average error']) )


# Clustered MTL
K = 4 # number of clusters
base.clustered_train(data_train, K = K, eta_B = eta, T = T)
base.results_baseline = base.evaluate(data_test, model = 'clustered')
print( 'Clustered MTL: {}'.format(base.results_baseline['average error']) )


# Low-rank MTL
K = 4 # rank
base.lowrank_train(data_train, K = K, eta_B = eta, eta_Z = eta, T = T)
base.results_baseline = base.evaluate(data_test, model = 'lowrank')
print( 'Low-rank MTL: {}'.format(base.results_baseline['average error']) )


Single-task learning: 0.025403028822667317
Data pooling: 0.03663898387884709
Clustered MTL: 0.022960429897410845
Low-rank MTL: 0.016609672691744015


In [5]:
# Multi-class logistic regression: direct classification of all 6 activities

[data_raw, _, _] = load(dataset = 'HAR', path = '', PCs = 100) # load data
[data_train, data_test] = split(data = data_raw, prop = prop, seed = seed) # train-test split
d = data_train[0][0].shape[1] # dimension
n_list = np.array( [data_train[0][j].shape[0] for j in range(m)] ) # list of sample sizes

In [6]:
# ARMUL
test = ARMUL('logistic', n_class = 6)

# Example: take lambda_j = 0.1 * np.sqrt(d / n_j) for all j in [m]
lbd = 0.1 * np.sqrt(d / n_list)

# Vanilla ARMUL
test.vanilla(data_train, lbd, eta_global = eta, eta_local = eta, T_global = T)
test.results = test.evaluate(data_test, model = 'vanilla')
print( 'Vanilla ARMUL: {}'.format(test.results['average error']) ) # average testing error


# Clustered ARMUL
K = 4 # number of clusters
test.clustered(data_train, lbd, K = K, eta_B = eta, eta_local = eta, T_global = T)
test.results = test.evaluate(data_test, model = 'clustered')
print( 'Clustered ARMUL: {}'.format(test.results['average error']) )


# Low-rank ARMUL
K = 4 # rank
test.lowrank(data_train, lbd, K = K, eta_B = eta, eta_Z = eta, eta_local = eta, T_global = T)
test.results = test.evaluate(data_test, model = 'lowrank')
print( 'Low-rank ARMUL: {}'.format(test.results['average error']) )



Vanilla ARMUL: 0.009770395701025891
Clustered ARMUL: 0.009281875915974597
Low-rank ARMUL: 0.006350757205666829


In [7]:
# Cross-validation
C_list = [0.05, 0.1, 0.2, 0.4]
lbd_list = [C * np.sqrt(d / n_list) for C in C_list]
n_fold = 5 # number of folds for cross-validation


print('Vanilla ARMUL') # Vanilla ARMUL
test.cv(data_train, lbd_list, model = 'vanilla', n_fold = 5, seed = seed, eta_global = eta, eta_local = eta, eta_B = eta, eta_Z = eta, T_global = T)
print( 'CV errors corresponding to all C\'s: {}'.format(test.errors_cv) )
test.results = test.evaluate(data_test, model = 'vanilla')
print( 'Average testing error of the final model: {}'.format(test.results['average error']) )



Vanilla ARMUL
CV errors corresponding to all C's: [0.00908288 0.00883736 0.00798768 0.00896062]
Average testing error of the final model: 0.009281875915974597


In [8]:
# Baselines
base = Baselines('logistic', n_class = 6)

# Single-task learning
base.STL_train(data_train, eta = eta, T = T)
base.results_baseline = base.evaluate(data_test, model = 'STL')
print( 'Single-task learning: {}'.format(base.results_baseline['average error']) )


# Data pooling
base.DP_train(data_train, eta = eta, T = T)
base.results_baseline = base.evaluate(data_test, model = 'DP')
print( 'Data pooling: {}'.format(base.results_baseline['average error']) )


# Clustered MTL
K = 4 # number of clusters
base.clustered_train(data_train, K = K, eta_B = eta, T = T)
base.results_baseline = base.evaluate(data_test, model = 'clustered')
print( 'Clustered MTL: {}'.format(base.results_baseline['average error']) )


# Low-rank MTL
K = 4 # rank
base.lowrank_train(data_train, K = K, eta_B = eta, eta_Z = eta, T = T)
base.results_baseline = base.evaluate(data_test, model = 'lowrank')
print( 'Low-rank MTL: {}'.format(base.results_baseline['average error']) )


Single-task learning: 0.019540791402051783
Data pooling: 0.0356619443087445
Clustered MTL: 0.024914509037616023
Low-rank MTL: 0.014167073766487542
